In [ ]:
import os
from PIL import Image
from glob import glob
import scipy.misc
from random import randrange, randint, uniform
import copy
import math 
%matplotlib inline

In [ ]:
from keras.applications.vgg16 import VGG16
# from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing import image
# from keras.optimizers import Adam
from keras.models import Sequential, Model
# from keras.layers import Dropout, Flatten, Dense
from keras.layers.core import Lambda
# from keras.layers.normalization import BatchNormalization
import keras.backend as K
import tensorflow as tf

In [ ]:
class ImageGen():
    def __init__(self):
        self.base_dir = os.getcwd()
        self.data_dir = os.path.join(self.base_dir, 'data')
        self.crop_dir = os.path.join(self.data_dir, 'crops')
        self.bg_dir = os.path.join(self.data_dir, 'bgs')
        self.gen_dir = os.path.join(self.data_dir, 'gen')
        self.ori_dir = os.path.join(self.data_dir, 'ori')
        self.ori_csv = os.path.join(self.ori_dir, 'orientations.csv')
        self.training_list = os.path.join(self.gen_dir, 'training_list.txt')